In [106]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [107]:
df_clean = pd.read_pickle('df_clean_final.pkl')
df_clean

,age,sex,bmi,children,smoker,charges,is_southwest,is_southeast,is_northwest,is_northeast,age_cat,18_35,36_50,51_70,71_200,bmi_cat,underweight,normal,overweight,obesity
0,19,0,27.900,0,1,16884.92400,1,0,0,0,18_35,1,0,0,0,overweight,0,0,1,0
1,18,1,33.770,1,0,1725.55230,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
2,28,1,33.000,3,0,4449.46200,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
3,33,1,22.705,0,0,21984.47061,0,0,1,0,18_35,1,0,0,0,normal,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,0,1,0,18_35,1,0,0,0,overweight,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830,0,0,1,0,36_50,0,1,0,0,obesity,0,0,0,1
1334,18,0,31.920,0,0,2205.98080,0,0,0,1,18_35,1,0,0,0,obesity,0,0,0,1
1335,18,0,36.850,0,0,1629.83350,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
1336,21,0,25.800,0,0,2007.94500,1,0,0,0,18_35,1,0,0,0,overweight,0,0,1,0


In [108]:
# import du csv
df = pd.read_csv('data/data.csv')

# selection des features et de la cible
X = df.drop('charges', axis=1)
y = df.charges

# selection des features et de la cible
X = df.drop('charges', axis=1)
y = df.charges

# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=0)

In [130]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer


# Séparation des colonnes numériques et catégorielles

num_col = ['bmi','age','children']



# pipeline pour les valeurs numériques
# cela permet de créer une succession d'étapes (ici il n'y en a qu'une)

my_num_pipe = make_pipeline(StandardScaler())

my_cat_pipe = make_pipeline(OneHotEncoder())

my_ordi_pipe = make_pipeline(OrdinalEncoder())
                            
# ColumnTransformer permet de faire des opérations sur des sélections de colonnes
preprocessor = make_column_transformer((my_num_pipe,num_col),
                                       (my_cat_pipe,['region']),
                                       (my_ordi_pipe,['sex','smoker']))


# affichage

tab = preprocessor.fit(df)

cols = tab.get_feature_names_out()

df_transform = pd.DataFrame(preprocessor.fit_transform(df), columns=cols)

df_transform.head()



,pipeline-1__bmi,pipeline-1__age,pipeline-1__children,pipeline-2__region_northeast,pipeline-2__region_northwest,pipeline-2__region_southeast,pipeline-2__region_southwest,pipeline-3__sex,pipeline-3__smoker
0,-0.453320,-1.438764,-0.908614,0.0,0.0,0.0,1.0,0.0,1.0
1,0.509621,-1.509965,-0.078767,0.0,0.0,1.0,0.0,1.0,0.0
2,0.383307,-0.797954,1.580926,0.0,0.0,1.0,0.0,1.0,0.0
3,-1.305531,-0.441948,-0.908614,0.0,1.0,0.0,0.0,1.0,0.0
4,-0.292556,-0.513149,-0.908614,0.0,1.0,0.0,0.0,1.0,0.0


In [129]:
from sklearn.model_selection import GridSearchCV

model= make_pipeline(preprocessor, ElasticNet())
model.fit(X_train, y_train)
model.score(X_test, y_test)

# Définir les valeurs à tester pour alpha et l1_ratio

param_grid = {'elasticnet__alpha': np.arange(0.1, 1,100),
              'elasticnet__l1_ratio': np.arange(0.1,1,0.01)}

# création du modèle

model= make_pipeline(preprocessor,PolynomialFeatures(degree=2),ElasticNet(max_iter=1000000))

# creation de l'objet GridSearch

grid_search = GridSearchCV(model,param_grid, cv=5)

# recherche des hyperparamètres et Afficher les meilleures valeurs trouvées

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)

print("Score : ", best_model.score(X_test, y_test))

# calcul RSME

rmse= np.sqrt(mean_squared_error(y_test,best_model.predict(X_test)))
print("RSME: ",rmse)


Meilleurs paramètres :  {'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9699999999999995}
Score :  0.8768692315377652
RSME:  4529.895850695082


## Regression Linéaire 

In [112]:

y=df_clean[['charges']]
X = df_clean[['bmi','age','smoker','sex','children']]

# regression lineaire, calcule R2, scatterplot

model = make_pipeline(preprocessor,LinearRegression())
model.fit(X_train,y_train)
model.score(X_train,y_train)
print("R2 (coefficient de détermination): ",model.score(X_test,y_test))


R2 (coefficient de détermination):  0.7918340172450387


## Regression Lasso

In [113]:
model= make_pipeline(preprocessor,Lasso())
model.fit(X_train, y_train)
model.score(X_test,y_test)


0.7917924840570529

## Regression Ridge

In [114]:
model= make_pipeline(preprocessor,Ridge())
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7915674076603373